In [1]:
import networkx as nx
import random
import heapq
import matplotlib.pyplot as plt
import numpy as np
import time
from tqdm import tqdm
import pandas as pd

In [9]:
def expected_density_v3(G, S, T):
    if G.number_of_edges() == 0:
        return 0
    probability_sum = sum(G[u][v]['weight'] * G[u][v]['probability'] for u, v in G.edges() if u in S and v in T)
    return probability_sum / (len(S) * len(T))**0.5


def plot_graph_v2(G):
    pos = nx.spring_layout(G)  # Tạo bố cục cho đồ thị

    # Lấy thuộc tính xác suất và trọng số của các cạnh
    edge_probabilities = nx.get_edge_attributes(G, 'probability')
    edge_weights = nx.get_edge_attributes(G, 'weight')

    # Kết hợp xác suất và trọng số vào nhãn cạnh
    edge_labels = {edge: f"{edge_weights[edge]:.2f}, {edge_probabilities[edge]:.2f}" for edge in G.edges()}

    # Vẽ đồ thị
    nx.draw(G, pos, with_labels=True, node_color='lightblue', edge_color='red', node_size=200, font_size=8, font_weight='bold')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=7)  # Hiển thị nhãn của các cạnh

    plt.title('Uncertain Weighted Directed Graph Visualization')
    plt.show()
    
def average_edge_probability(G):
    """ Tính xác suất trung bình của các cạnh trong đồ thị G. """
    total_probability = sum(nx.get_edge_attributes(G, 'probability').values())
    num_edges = G.number_of_edges()
    return total_probability / num_edges if num_edges > 0 else 0



def calculate_f_beta_v3(G_S, beta, S, T):
    num_edges = G_S.number_of_edges()    
    if len(S) > 0 and len(T) > 0:
        f_beta = (weighted_average_edge_probability(G_S) - beta) * (num_edges / (len(S) * len(T))**0.5)
    else:
        f_beta = 0
    return f_beta
    

def weighted_average_edge_probability(G):
    total_weighted_probability = sum(G[u][v]['weight'] * G[u][v]['probability'] for u, v in G.edges())  
    num_edges = G.number_of_edges()
    return total_weighted_probability / num_edges if num_edges > 0 else 0

def surplus_degree_out(G, v, beta):
    return sum(G[v][w]['weight'] *G[v][w]['probability'] - beta for w in G.successors(v) if w in G)

def surplus_degree_in(G, v, beta):
    return sum(G[u][v]['weight'] *G[u][v]['probability'] - beta for u in G.predecessors(v) if u in G)


def initialize_priority_queue_out(G, beta):
    """ Khởi tạo hàng đợi ưu tiên với bậc dư thừa cho mỗi đỉnh """
    priority_queue = []
    for v in G.nodes():
        sd = surplus_degree_out(G, v, beta)
        heapq.heappush(priority_queue, (sd, v))
        
    return priority_queue

def initialize_priority_queue_in(G, beta):
    """ Khởi tạo hàng đợi ưu tiên với bậc dư thừa cho mỗi đỉnh """
    priority_queue = []
    for v in G.nodes():
        sd = surplus_degree_in(G, v, beta)
        heapq.heappush(priority_queue, (sd, v))
        
    return priority_queue


def calculate_edge_density_v3(G):
    """Tính mật độ cạnh kỳ vọng của đồ thị G. τ"""
    num_vertices = len(G.nodes())
    num_possible_edges = num_vertices * (num_vertices - 1) if num_vertices > 1 else 1
    sum_weighted_probabilities = sum(G[u][v]['probability'] for u, v in G.edges())
    return sum_weighted_probabilities / num_possible_edges

def expected_density_v3(G, S, T):
    if G.number_of_edges() == 0:
        return 0
    probability_sum = sum(G[u][v]['weight'] * G[u][v]['probability'] for u, v in G.edges() if u in S and v in T)
    return probability_sum / (len(S) * len(T))**0.5

def trong_so_trung_binh_canh(G):
    """ Tính xác suất trung bình của các cạnh trong đồ thị G. """
    total_probability = sum(nx.get_edge_attributes(G, 'weight').values())
    num_edges = G.number_of_edges()
    return total_probability / num_edges if num_edges > 0 else 0

def greedy_average_surplus_degree(G, beta):
    H = G.copy()
    S_temp = set(H.nodes())
    T_temp = set(H.nodes())
    best_subgraph = H.copy()
    best_f_beta = calculate_f_beta_v3(best_subgraph, beta, S_temp, T_temp)

    priority_queue_out = initialize_priority_queue_out(H, beta)
    priority_queue_in = initialize_priority_queue_in(H, beta)

    with tqdm(total=H.number_of_nodes() - 2, desc="Processing", unit="node") as pbar:
        while S_temp and T_temp:
            while priority_queue_out:
                _, i_min = heapq.heappop(priority_queue_out)
                if i_min in H:
                    break

            while priority_queue_in:
                _, j_min = heapq.heappop(priority_queue_in)
                if j_min in H:
                    break

            d_S = surplus_degree_out(H, i_min, beta) if i_min in H else float('inf')
            d_T = surplus_degree_in(H, j_min, beta) if j_min in H else float('inf')
            
            if d_S <= d_T:
                S_temp.remove(i_min)
                H.remove_node(i_min)
            else:
                T_temp.remove(j_min)
                H.remove_node(j_min)

            current_f_beta = calculate_f_beta_v3(H, beta, S_temp, T_temp)
            
            S_temp = {u for u in S_temp if u in H}
            T_temp = {v for v in T_temp if v in H}
            
            if current_f_beta > best_f_beta:
                best_f_beta = current_f_beta
                best_subgraph = H.copy()  # Lưu bản sao của H với mật độ cao nhất

            # Cập nhật lại hàng đợi ưu tiên
            priority_queue_out = initialize_priority_queue_out(H, beta)
            priority_queue_in = initialize_priority_queue_in(H, beta)
            
            pbar.update(1)  # Cập nhật tiến trình mỗi khi một đỉnh được loại bỏ
    
    return best_subgraph


In [10]:
# Đọc đồ thị từ file và gán xác suất ngẫu nhiên cho các cạnh
random.seed(7)
G = nx.DiGraph()

with open('data/facebook_combined.txt', 'r') as file:
    for line in file:
        if line.startswith('#'):
            continue
        source, target = map(int, line.split())
        probability = random.random()  # Xác suất ngẫu nhiên giữa 0 và 1
        weight = random.random()
        if random.choice([True, False]):
            G.add_edge(source, target, probability=probability, weight=weight)
#         G.add_edge(source, target, probability=probability, weight=weight)
        

print("Number of nodes:", G.number_of_nodes())
print("Number of edges:", G.number_of_edges())

# Xác định các tập S và T từ các cạnh của đồ thị
S = set()
T = set()
for u, v in G.edges():
    S.add(u)
    T.add(v)

# print("Set S:", S)
# print("Set T:", T)


# Lấy các giá trị xác suất từ các cạnh
probabilities = [G[u][v]['probability'] for u, v in G.edges()]
weights = [G[u][v]['weight'] for u, v in G.edges()]
# Tính giá trị trung bình
mean_probability = np.mean(probabilities)
mean_weight = np.mean(weights)


# Tính độ lệch chuẩn
std_deviation = np.std(probabilities)

    
print("Giá trị cạnh trung bình của đồ thị :", mean_probability)
print("Độ lệch chuẩn của của  đồ thị :", std_deviation)
print("Trọng số cạnh trung bình của đồ thị là:",mean_weight)




Number of nodes: 3970
Number of edges: 44364
Giá trị cạnh trung bình của đồ thị : 0.5010087588685139
Độ lệch chuẩn của của  đồ thị : 0.2893428018764357
Trọng số cạnh trung bình của đồ thị là: 0.5028146226860798


In [7]:
beta = 0.5

ObsEmailEuCore01 = greedy_average_surplus_degree(G,beta)
# Xác định các tập S và T từ các cạnh của đồ thị
S = set()
T = set()
for u, v in ObsEmailEuCore01.edges():
    S.add(u)
    T.add(v)

# Lấy các giá trị xác suất từ các cạnh
probabilities = [ObsEmailEuCore01[u][v]['probability'] for u, v in ObsEmailEuCore01.edges()]
weights = [ObsEmailEuCore01[u][v]['weight'] for u, v in ObsEmailEuCore01.edges()]

# Tính giá trị trung bình
mean_probability = np.mean(probabilities)
mean_weight = np.mean(weights)

# Tính độ lệch chuẩn
std_deviation = np.std(probabilities)
print("Mật độ kì vọng của đồ thị con do thuật toán obs sinh ra là:", expected_density_v3(ObsEmailEuCore01, S, T))
print("Số đỉnh trong đồ thị con của thuật toán obs  là:", ObsEmailEuCore01.number_of_nodes())
print("Số cạnh trong đồ thị con của thuật toán obs  là:", ObsEmailEuCore01.number_of_edges())
print("Mật độ kì cạnh vọng của thuật toán obs  là:",calculate_edge_density_v3(ObsEmailEuCore01))
print("Xác xuất cạnh trung bình của thuật toán obs là:",average_edge_probability(ObsEmailEuCore01))
print(" trọng số cạnh trung bình của thuật toán obs là:",mean_weight)
print("Độ lệch chuẩn của thuật toán obs:", std_deviation)

Processing: 3970node [05:55, 11.15node/s]                                                                              


Mật độ kì vọng của đồ thị con do thuật toán obs sinh ra là: 0.8687298904722905
Số đỉnh trong đồ thị con của thuật toán obs  là: 374
Số cạnh trong đồ thị con của thuật toán obs  là: 96
Mật độ kì cạnh vọng của thuật toán obs  là: 0.0004850953556069801
Xác xuất cạnh trung bình của thuật toán obs là: 0.8321176648986874
 trọng số cạnh trung bình của thuật toán obs là: 0.8498167254552212
Độ lệch chuẩn của thuật toán obs: 0.12608397804646532


In [11]:
beta = 0.5 * trong_so_trung_binh_canh(G)

ObsEmailEuCore01 = greedy_average_surplus_degree(G,beta)
# Xác định các tập S và T từ các cạnh của đồ thị
S = set()
T = set()
for u, v in ObsEmailEuCore01.edges():
    S.add(u)
    T.add(v)

# Lấy các giá trị xác suất từ các cạnh
probabilities = [ObsEmailEuCore01[u][v]['probability'] for u, v in ObsEmailEuCore01.edges()]
weights = [ObsEmailEuCore01[u][v]['weight'] for u, v in ObsEmailEuCore01.edges()]

# Tính giá trị trung bình
mean_probability = np.mean(probabilities)
mean_weight = np.mean(weights)

# Tính độ lệch chuẩn
std_deviation = np.std(probabilities)
print("Mật độ kì vọng của đồ thị con do thuật toán obs sinh ra là:", expected_density_v3(ObsEmailEuCore01, S, T))
print("Số đỉnh trong đồ thị con của thuật toán obs  là:", ObsEmailEuCore01.number_of_nodes())
print("Số cạnh trong đồ thị con của thuật toán obs  là:", ObsEmailEuCore01.number_of_edges())
print("Mật độ kì cạnh vọng của thuật toán obs  là:",calculate_edge_density_v3(ObsEmailEuCore01))
print("Xác xuất cạnh trung bình của thuật toán obs là:",average_edge_probability(ObsEmailEuCore01))
print(" trọng số cạnh trung bình của thuật toán obs là:",mean_weight)
print("Độ lệch chuẩn của thuật toán obs:", std_deviation)

Processing:   1%|▎                                                                 | 20/3968 [00:09<32:07,  2.05node/s]


KeyboardInterrupt: 

In [ ]:
beta = 0.1 * trong_so_trung_binh_canh(G)

ObsEmailEuCore01 = greedy_average_surplus_degree(G,beta)
# Xác định các tập S và T từ các cạnh của đồ thị
S = set()
T = set()
for u, v in ObsEmailEuCore01.edges():
    S.add(u)
    T.add(v)

# Lấy các giá trị xác suất từ các cạnh
probabilities = [ObsEmailEuCore01[u][v]['probability'] for u, v in ObsEmailEuCore01.edges()]
weights = [ObsEmailEuCore01[u][v]['weight'] for u, v in ObsEmailEuCore01.edges()]

# Tính giá trị trung bình
mean_probability = np.mean(probabilities)
mean_weight = np.mean(weights)

# Tính độ lệch chuẩn
std_deviation = np.std(probabilities)
print("Mật độ kì vọng của đồ thị con do thuật toán obs sinh ra là:", expected_density_v3(ObsEmailEuCore01, S, T))
print("Số đỉnh trong đồ thị con của thuật toán obs  là:", ObsEmailEuCore01.number_of_nodes())
print("Số cạnh trong đồ thị con của thuật toán obs  là:", ObsEmailEuCore01.number_of_edges())
print("Mật độ kì cạnh vọng của thuật toán obs  là:",calculate_edge_density_v3(ObsEmailEuCore01))
print("Xác xuất cạnh trung bình của thuật toán obs là:",average_edge_probability(ObsEmailEuCore01))
print(" trọng số cạnh trung bình của thuật toán obs là:",mean_weight)
print("Độ lệch chuẩn của thuật toán obs:", std_deviation)
beta = 0.2 * trong_so_trung_binh_canh(G)

ObsEmailEuCore01 = greedy_average_surplus_degree(G,beta)
# Xác định các tập S và T từ các cạnh của đồ thị
S = set()
T = set()
for u, v in ObsEmailEuCore01.edges():
    S.add(u)
    T.add(v)

# Lấy các giá trị xác suất từ các cạnh
probabilities = [ObsEmailEuCore01[u][v]['probability'] for u, v in ObsEmailEuCore01.edges()]
weights = [ObsEmailEuCore01[u][v]['weight'] for u, v in ObsEmailEuCore01.edges()]

# Tính giá trị trung bình
mean_probability = np.mean(probabilities)
mean_weight = np.mean(weights)

# Tính độ lệch chuẩn
std_deviation = np.std(probabilities)
print("Mật độ kì vọng của đồ thị con do thuật toán obs sinh ra là:", expected_density_v3(ObsEmailEuCore01, S, T))
print("Số đỉnh trong đồ thị con của thuật toán obs  là:", ObsEmailEuCore01.number_of_nodes())
print("Số cạnh trong đồ thị con của thuật toán obs  là:", ObsEmailEuCore01.number_of_edges())
print("Mật độ kì cạnh vọng của thuật toán obs  là:",calculate_edge_density_v3(ObsEmailEuCore01))
print("Xác xuất cạnh trung bình của thuật toán obs là:",average_edge_probability(ObsEmailEuCore01))
print(" trọng số cạnh trung bình của thuật toán obs là:",mean_weight)
print("Độ lệch chuẩn của thuật toán obs:", std_deviation)

beta = 0.3 * trong_so_trung_binh_canh(G)

ObsEmailEuCore01 = greedy_average_surplus_degree(G,beta)
# Xác định các tập S và T từ các cạnh của đồ thị
S = set()
T = set()
for u, v in ObsEmailEuCore01.edges():
    S.add(u)
    T.add(v)

# Lấy các giá trị xác suất từ các cạnh
probabilities = [ObsEmailEuCore01[u][v]['probability'] for u, v in ObsEmailEuCore01.edges()]
weights = [ObsEmailEuCore01[u][v]['weight'] for u, v in ObsEmailEuCore01.edges()]

# Tính giá trị trung bình
mean_probability = np.mean(probabilities)
mean_weight = np.mean(weights)

# Tính độ lệch chuẩn
std_deviation = np.std(probabilities)
print("Mật độ kì vọng của đồ thị con do thuật toán obs sinh ra là:", expected_density_v3(ObsEmailEuCore01, S, T))
print("Số đỉnh trong đồ thị con của thuật toán obs  là:", ObsEmailEuCore01.number_of_nodes())
print("Số cạnh trong đồ thị con của thuật toán obs  là:", ObsEmailEuCore01.number_of_edges())
print("Mật độ kì cạnh vọng của thuật toán obs  là:",calculate_edge_density_v3(ObsEmailEuCore01))
print("Xác xuất cạnh trung bình của thuật toán obs là:",average_edge_probability(ObsEmailEuCore01))
print(" trọng số cạnh trung bình của thuật toán obs là:",mean_weight)
print("Độ lệch chuẩn của thuật toán obs:", std_deviation)
beta = 0.4 * trong_so_trung_binh_canh(G)

ObsEmailEuCore01 = greedy_average_surplus_degree(G,beta)
# Xác định các tập S và T từ các cạnh của đồ thị
S = set()
T = set()
for u, v in ObsEmailEuCore01.edges():
    S.add(u)
    T.add(v)

# Lấy các giá trị xác suất từ các cạnh
probabilities = [ObsEmailEuCore01[u][v]['probability'] for u, v in ObsEmailEuCore01.edges()]
weights = [ObsEmailEuCore01[u][v]['weight'] for u, v in ObsEmailEuCore01.edges()]

# Tính giá trị trung bình
mean_probability = np.mean(probabilities)
mean_weight = np.mean(weights)

# Tính độ lệch chuẩn
std_deviation = np.std(probabilities)
print("Mật độ kì vọng của đồ thị con do thuật toán obs sinh ra là:", expected_density_v3(ObsEmailEuCore01, S, T))
print("Số đỉnh trong đồ thị con của thuật toán obs  là:", ObsEmailEuCore01.number_of_nodes())
print("Số cạnh trong đồ thị con của thuật toán obs  là:", ObsEmailEuCore01.number_of_edges())
print("Mật độ kì cạnh vọng của thuật toán obs  là:",calculate_edge_density_v3(ObsEmailEuCore01))
print("Xác xuất cạnh trung bình của thuật toán obs là:",average_edge_probability(ObsEmailEuCore01))
print(" trọng số cạnh trung bình của thuật toán obs là:",mean_weight)
print("Độ lệch chuẩn của thuật toán obs:", std_deviation)
beta = 0.6 * trong_so_trung_binh_canh(G)

ObsEmailEuCore01 = greedy_average_surplus_degree(G,beta)
# Xác định các tập S và T từ các cạnh của đồ thị
S = set()
T = set()
for u, v in ObsEmailEuCore01.edges():
    S.add(u)
    T.add(v)

# Lấy các giá trị xác suất từ các cạnh
probabilities = [ObsEmailEuCore01[u][v]['probability'] for u, v in ObsEmailEuCore01.edges()]
weights = [ObsEmailEuCore01[u][v]['weight'] for u, v in ObsEmailEuCore01.edges()]

# Tính giá trị trung bình
mean_probability = np.mean(probabilities)
mean_weight = np.mean(weights)

# Tính độ lệch chuẩn
std_deviation = np.std(probabilities)
print("Mật độ kì vọng của đồ thị con do thuật toán obs sinh ra là:", expected_density_v3(ObsEmailEuCore01, S, T))
print("Số đỉnh trong đồ thị con của thuật toán obs  là:", ObsEmailEuCore01.number_of_nodes())
print("Số cạnh trong đồ thị con của thuật toán obs  là:", ObsEmailEuCore01.number_of_edges())
print("Mật độ kì cạnh vọng của thuật toán obs  là:",calculate_edge_density_v3(ObsEmailEuCore01))
print("Xác xuất cạnh trung bình của thuật toán obs là:",average_edge_probability(ObsEmailEuCore01))
print(" trọng số cạnh trung bình của thuật toán obs là:",mean_weight)
print("Độ lệch chuẩn của thuật toán obs:", std_deviation)
beta = 0.7 * trong_so_trung_binh_canh(G)

ObsEmailEuCore01 = greedy_average_surplus_degree(G,beta)
# Xác định các tập S và T từ các cạnh của đồ thị
S = set()
T = set()
for u, v in ObsEmailEuCore01.edges():
    S.add(u)
    T.add(v)

# Lấy các giá trị xác suất từ các cạnh
probabilities = [ObsEmailEuCore01[u][v]['probability'] for u, v in ObsEmailEuCore01.edges()]
weights = [ObsEmailEuCore01[u][v]['weight'] for u, v in ObsEmailEuCore01.edges()]

# Tính giá trị trung bình
mean_probability = np.mean(probabilities)
mean_weight = np.mean(weights)

# Tính độ lệch chuẩn
std_deviation = np.std(probabilities)
print("Mật độ kì vọng của đồ thị con do thuật toán obs sinh ra là:", expected_density_v3(ObsEmailEuCore01, S, T))
print("Số đỉnh trong đồ thị con của thuật toán obs  là:", ObsEmailEuCore01.number_of_nodes())
print("Số cạnh trong đồ thị con của thuật toán obs  là:", ObsEmailEuCore01.number_of_edges())
print("Mật độ kì cạnh vọng của thuật toán obs  là:",calculate_edge_density_v3(ObsEmailEuCore01))
print("Xác xuất cạnh trung bình của thuật toán obs là:",average_edge_probability(ObsEmailEuCore01))
print(" trọng số cạnh trung bình của thuật toán obs là:",mean_weight)
print("Độ lệch chuẩn của thuật toán obs:", std_deviation)

beta = 0.8 * trong_so_trung_binh_canh(G)

ObsEmailEuCore01 = greedy_average_surplus_degree(G,beta)
# Xác định các tập S và T từ các cạnh của đồ thị
S = set()
T = set()
for u, v in ObsEmailEuCore01.edges():
    S.add(u)
    T.add(v)

# Lấy các giá trị xác suất từ các cạnh
probabilities = [ObsEmailEuCore01[u][v]['probability'] for u, v in ObsEmailEuCore01.edges()]
weights = [ObsEmailEuCore01[u][v]['weight'] for u, v in ObsEmailEuCore01.edges()]

# Tính giá trị trung bình
mean_probability = np.mean(probabilities)
mean_weight = np.mean(weights)

# Tính độ lệch chuẩn
std_deviation = np.std(probabilities)
print("Mật độ kì vọng của đồ thị con do thuật toán obs sinh ra là:", expected_density_v3(ObsEmailEuCore01, S, T))
print("Số đỉnh trong đồ thị con của thuật toán obs  là:", ObsEmailEuCore01.number_of_nodes())
print("Số cạnh trong đồ thị con của thuật toán obs  là:", ObsEmailEuCore01.number_of_edges())
print("Mật độ kì cạnh vọng của thuật toán obs  là:",calculate_edge_density_v3(ObsEmailEuCore01))
print("Xác xuất cạnh trung bình của thuật toán obs là:",average_edge_probability(ObsEmailEuCore01))
print(" trọng số cạnh trung bình của thuật toán obs là:",mean_weight)
print("Độ lệch chuẩn của thuật toán obs:", std_deviation)